<div style="text-align:center; font-family:Georgia;">
    <h1>Financial Planner for Retirement</h1>
</div>

 <img src="https://aginginplace.com/wp-content/uploads/2018/11/AdobeStock_128753653-e1541193221829.jpeg" 
    width = 1000 height = 10>

<div style="font-family:Calibri;">
<h3> Lilian Bonilla & Andrea Wupuy <br>
     December 2023 </h3>
</div>

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Hi there! Hope the retiree life is treating you well! We know how important managing personal finances is. This calculator here to help you forecast your expenses and be aware when you will run out of cash. With a few simple clicks, you will be an expert at managing your finances.
<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Let's get started!
<br>

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<div style="font-family:Georgia;">
    <h3>1. Information Set up: Please do not modify </h3>
</div>

<div style="font-family:Georgia;">
    <h4>a. Import Libraries and Warnings:</h4>

In [169]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore") 

In [170]:
# Importing libraries
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # create graphs
import seaborn as sns # create graphs
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import os
import csv

In [171]:
# Random Seed
np.random.seed(42)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<div style="font-family:Georgia;">
    <h4>b. Loading Files:</h4>

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; The calculator feeds up on 5 files: inflation, RMD distribution, taxes for MA, CA and Federation. The taxes files should all have the same structure so that the calculator reads it properly.
<br>

In [172]:
def load_data(filename):
    data_path = "./data/"
    df_data = pd.read_csv(data_path + filename, delimiter=';')
    return df_data.copy()

# Load data for Inflation, Tax CA, Tax MA, Tax Federation, and RMD
inflation_df_data = load_data("inflation.csv")
taxCA_df_data = load_data("tax_CA.csv")
taxMA_df_data = load_data("tax_MA.csv")
taxfed_df_data = load_data("tax_federation.csv")
rmd_df_data = load_data("RMD.csv")

# Create working copies of the dataframes for analysis
inflation_df = inflation_df_data.copy()
taxCA_df = taxCA_df_data.copy()
taxMA_df = taxMA_df_data.copy()
taxfed_df = taxfed_df_data.copy()
rmd_df = rmd_df_data.copy()

In [173]:
taxCA_df['Up to'] = taxCA_df['Up to'].str.replace(',', '').astype('float64')
taxCA_df['Over'] = taxCA_df['Over'].str.replace(',', '').astype('float64')
taxCA_df['Base tax $'] = taxCA_df['Base tax $'].replace('[\$,]', '', regex=True).astype('float64')
taxCA_df['% on excess'] = taxCA_df['% on excess'].replace(',', '', regex=True).str.rstrip('%').astype('float') / 100.0
taxfed_df['Up to'] = taxfed_df['Up to'].str.replace(',', '').astype('float64')
taxfed_df['Over'] = taxfed_df['Over'].str.replace(',', '').astype('float64')
taxfed_df['Base tax $'] = taxfed_df['Base tax $'].replace('[\$,]', '', regex=True).astype('float64')
taxfed_df['% on excess'] = taxfed_df['% on excess'].replace(',', '', regex=True).str.rstrip('%').astype('float') / 100.0
taxMA_df['% on excess'] = taxMA_df['% on excess'].replace(',', '', regex=True).str.rstrip('%').astype('float') / 100.0

In [174]:
# Set the display option to show floats without scientific notation
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

# Assuming balances_df_ira is your DataFrame
pd.set_option('display.max_rows', None)

# Resetting the option to default
pd.set_option('display.max_rows', pd.get_option('display.max_rows'))

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<div style="font-family:Georgia;">
    <h4>c. Defining Functions:</h4>

In [175]:
def calculate_taxes(income, tax_brackets_df):
    total_taxes = 0

    for index, row in tax_brackets_df.iterrows():
        lower_limit = row['Over']
        upper_limit = row['Up to']
        base_tax = row['Base tax $']
        excess_rate = row['% on excess']

        if np.isnan(upper_limit):  # Check if upper_limit is NaN
            upper_limit = np.inf  # Set it to positive infinity for an open upper limit

        # Commenting out the print statements
        # print(f"Income: {income}, Lower Limit: {lower_limit}, Upper Limit: {upper_limit}, Base Tax: {base_tax}, Excess Rate: {excess_rate}")

        if income <= lower_limit:
            break
        elif income <= upper_limit:
            total_taxes += base_tax + (income - lower_limit) * excess_rate
            break
        else:
            continue

    # Commenting out the print statement
    # print(f"Total Taxes: {total_taxes}")
    return total_taxes


In [176]:
# Function to calculate RMD for a given age
def calculate_rmd(age, rmd_data):
    age_str = str(age)
    if age_str in rmd_data:
        return rmd_data[age_str]['Distribution Period']
    else:
        return 0

In [177]:
def calculate_rmd(age, initial_balance_ira_current, rmd_df):
    row_age = rmd_df[rmd_df['Age'] == age]
    if not row_age.empty:
        distribution_period = row_age['Distribution Period'].values[0]
        rmd = initial_balance_ira_current / distribution_period
        return rmd
    else:
        #print(f"None")
        return 0

In [178]:
def calculate_real_monthly_expenses(annual_expenses, year, inflation_df_exp):
    # Assuming you want inflation for the current year
    real_monthly_expenses = annual_expenses

    # Print the year to check its value
    #print(f"Year in calculate_real_monthly_expenses: {real_monthly_expenses}")

    # Check if the year exists in the DataFrame
    if year in inflation_df_exp['Year'].values:
        # Get the inflation rate for the specified year
        inflation_rate_str = inflation_df_exp.loc[inflation_df_exp['Year'] == year, 'Annual'].values[0]

        # Remove the percentage sign and convert to float
        inflation_rate = float(inflation_rate_str.strip('%')) / 100

        # Update real monthly expenses with inflation
        real_monthly_expenses *= (inflation_rate)

    return real_monthly_expenses

In [179]:
def compare_tax_states(initial_balance_savings, initial_balance_ira, savings_interest_rate, ira_interest_rate, current_age, annual_expenses, year_value, calculation_age, inflation_data, rmd_data, tax_state1, tax_state2, tax_fed):
    # Calculate cash flow for the first tax_state dataframe
    cash_flow_df1 = calculate_cash_flow(initial_balance_savings, initial_balance_ira, savings_interest_rate, ira_interest_rate, current_age, annual_expenses, year_value, calculation_age, inflation_data, rmd_data, tax_state1, tax_fed)

    # Calculate total taxes for the first tax_state dataframe
    total_taxes1 = cash_flow_df1['Total Taxes'].sum()

    # Calculate cash flow for the second tax_state dataframe
    cash_flow_df2 = calculate_cash_flow(initial_balance_savings, initial_balance_ira, savings_interest_rate, ira_interest_rate, current_age, annual_expenses, year_value, calculation_age, inflation_data, rmd_data, tax_state2, tax_fed)

    # Calculate total taxes for the second tax_state dataframe
    total_taxes2 = cash_flow_df2['Total Taxes'].sum()

    if total_taxes1 < total_taxes2:
        print("\n")
        print("\033[1mThe state where you pay less taxes based on your input is California:\033[0m")
        print(f"Total Taxes for California ${total_taxes1:,.2f}")
        print(f"Will save in taxes: ${total_taxes2 - total_taxes1:,.2f}")
        return cash_flow_df1
    else:
        print("\n")
        print("\033[1mThe state where you pay less taxes based on your input is Massachussets:\033[0m")
        print(f"Total Taxes for Massachusetts: ${total_taxes2:,.2f}")
        print(f"Will save in taxes: ${total_taxes1 - total_taxes2:,.2f}")
        return cash_flow_df2

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<div style="font-family:Georgia;">
    <h1>2.. Retirement Calculator: Insert Inputs </h1>
</div>

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; The first step when creating an acurrate cashflow that fits your needs is tha tyou provide the most accurate inputs. The calculator requires 8 inputs from you to work.
<br>

In [180]:
def calculate_cash_flow(initial_balance_savings, initial_balance_ira, savings_interest_rate, ira_interest_rate, current_age, annual_expenses, year_value, calculation_age, inflation_data, rmd_data, tax_state, tax_fed):   
    # Initialize lists to store results
    years = []
    age_list = []
    calculation_age_list = []
    initial_balance_savings_list = []
    initial_balance_ira_list = []
    savings_interest_list = []
    ira_interest_list = []
    total_expenses_list = []
    taxable_income_list = []
    total_taxes_list = []
    final_balance_savings_list = []
    final_balance_ira_list = []
    rmd_distribution_list = []

    # Initialize variables
    total_expenses = annual_expenses  # Initial value
    #age = current_age
    
    age = current_age - 1
    max_age_condition_met = None
    # Iterate over the years
    for year in range(2023, 2023 + calculation_age - age + 1):  # Assuming an 80-year projection
            
        # Calculate RMD for the current age
          # Assuming current_age represents the age at year 2023
            # Increment age for the next iteration
        age += 1
        
        # Calculate real monthly expenses with inflation
        real_monthly_expenses = calculate_real_monthly_expenses(annual_expenses, year, inflation_data)

        # Initial balances (using the parameters passed to the function or the final balances from the previous year)
        initial_balance_savings_current = final_balance_savings_list[-1] if year > 2023 and final_balance_savings_list else initial_balance_savings
        initial_balance_ira_current = final_balance_ira_list[-1] if year > 2023 and final_balance_ira_list else initial_balance_ira

        # Update balances and income for the next year
        savings_interest = initial_balance_savings_current * savings_interest_rate
        ira_interest = initial_balance_ira_current * ira_interest_rate
        total_expenses += real_monthly_expenses# Cumulative real monthly expenses with inflation
        #taxable_income = savings_interest + total_expenses
        rmd_distribution = calculate_rmd(age, initial_balance_ira_current, rmd_data)  
        
        if rmd_distribution != 0 and rmd_distribution < total_expenses: 
            taxable_income = savings_interest + total_expenses
        elif rmd_distribution != 0 and rmd_distribution > total_expenses:
            taxable_income = savings_interest + rmd_distribution
        else:
            taxable_income = savings_interest
        
        taxes = calculate_taxes(taxable_income, tax_state)
        federation = calculate_taxes(taxable_income, tax_fed)                   
        total_taxes = taxes + federation
        
        # Calculate final balances
        final_balance_savings = initial_balance_savings_current + savings_interest - total_expenses - total_taxes
        final_balance_ira = initial_balance_ira_current + ira_interest - total_expenses - total_taxes
        
        condition_rmd = 1 if rmd_distribution > total_expenses else 0
        condition_savings = 1 if final_balance_savings < total_expenses else 0
        # Check if the final balance of the savings account is less than the expenses
        #if (final_balance_savings := initial_balance_savings_current + savings_interest - total_expenses - total_taxes) < 0:
            # If it is, deduct the remaining expenses from the IRA account
            #final_balance_ira = initial_balance_ira_current + ira_interest - total_expenses - total_taxes
            #final_balance_savings = initial_balance_savings_current + savings_interest
        #else:
            #final_balance_ira = initial_balance_ira_current + ira_interest

        if ((final_balance_savings := initial_balance_savings_current + savings_interest - total_expenses - total_taxes) < 0
        and rmd_distribution < total_expenses):
            final_balance_ira = initial_balance_ira_current + ira_interest - total_expenses - total_taxes
            final_balance_savings = initial_balance_savings_current + savings_interest
        elif ((final_balance_savings := initial_balance_savings_current + savings_interest - total_expenses - total_taxes) < 0
            and rmd_distribution >= total_expenses):
            final_balance_ira = initial_balance_ira_current + ira_interest - rmd_distribution - total_taxes
            final_balance_savings = initial_balance_savings_current + savings_interest + (rmd_distribution - total_expenses )
        elif ((final_balance_savings := initial_balance_savings_current + savings_interest - total_expenses - total_taxes) > 0
            and rmd_distribution >= total_expenses ):
            final_balance_ira = initial_balance_ira_current + ira_interest - rmd_distribution
            final_balance_savings = initial_balance_savings_current + savings_interest + rmd_distribution - total_taxes - total_expenses
        elif ((final_balance_savings := initial_balance_savings_current + savings_interest - total_expenses - total_taxes) > 0
            and rmd_distribution < total_expenses ):
            final_balance_ira = initial_balance_ira_current + ira_interest - rmd_distribution
            final_balance_savings = initial_balance_savings_current + savings_interest - total_expenses - total_taxes + rmd_distribution            
        else:
           0            

        if final_balance_ira <= 0:
            break

        # Store results in lists
        years.append(year)
        age_list.append(age)
        calculation_age_list.append(list)
        initial_balance_savings_list.append(initial_balance_savings_current)
        initial_balance_ira_list.append(initial_balance_ira_current)
        savings_interest_list.append(savings_interest)
        ira_interest_list.append(ira_interest)
        total_expenses_list.append(total_expenses)
        taxable_income_list.append(taxable_income)
        total_taxes_list.append(total_taxes)
        final_balance_savings_list.append(final_balance_savings)
        final_balance_ira_list.append(final_balance_ira)
        rmd_distribution_list.append(rmd_distribution)

    if final_balance_ira <= total_expenses:
        last_age_before_break = age - 1
        print("\n")
        print("You can live with your IRA account untill:", last_age_before_break)
    else:
        print("\n")
        print("You still have funds in your IRA")    
      
    # Create a DataFrame
    df_balance = pd.DataFrame({
        'Year': years,
        'Age': age_list,
        'Initial Balance Savings': initial_balance_savings_list,
        'Savings Interest': savings_interest_list,
        'Final Balance Savings': final_balance_savings_list,
        'Initial Balance IRA': initial_balance_ira_list,
        'IRA Interest': ira_interest_list,
        'Final Balance IRA': final_balance_ira_list,
        'Total Expenses': total_expenses_list,
        'Total Taxes': total_taxes_list,
        'RMD Distribution': rmd_distribution_list,
    })

    decreasing_age = None
    for i in range(1, len(df_balance)):
        if df_balance['Final Balance IRA'].iloc[i] < df_balance['Final Balance IRA'].iloc[i - 1]:
            decreasing_age = df_balance['Age'].iloc[i]
            break
    print("You can live with your SAVINGS account untill", decreasing_age)        
    # Print the DataFrame
    
    flow_df = pd.DataFrame(df_balance)
    return flow_df

inflation_data = inflation_df
rmd_data = rmd_df
tax_state1 = taxCA_df
tax_state2 = taxMA_df
tax_fed = taxfed_df

initial_balance_savings = float(input("Enter Total Savings in USD: "))
initial_balance_ira = float(input("Enter Total IRA Balance in USD: "))
savings_interest_rate = float(input("Enter Annual Interest Rate for Savings (Decimal): "))
ira_interest_rate = float(input("Enter Annual Interest Rate for IRA (Decimal): "))
current_age = int(input("Enter Current Age: "))
calculation_age = int(input("Enter Future Age:"))
year_value = int(input("Current Year:"))
annual_expenses = float(input("Enter Annual Expenses: "))



result_df = compare_tax_states(initial_balance_savings, initial_balance_ira, savings_interest_rate, ira_interest_rate, current_age, annual_expenses, year_value, calculation_age, inflation_data, rmd_data, tax_state1, tax_state2, tax_fed)
result_df


Enter Total Savings in USD: 580000
Enter Total IRA Balance in USD: 400000
Enter Annual Interest Rate for Savings (Decimal): 0.06
Enter Annual Interest Rate for IRA (Decimal): 0.08
Enter Current Age: 62
Enter Future Age:82
Current Year:2023
Enter Annual Expenses: 55000


You still have funds in your IRA
You can live with your SAVINGS account untill 74


You still have funds in your IRA
You can live with your SAVINGS account untill 73


The state where you pay less taxes based on your input is California:
Total Taxes for California $180,970.13
Will save in taxes: $27,539.26


,Year,Age,Initial Balance Savings,Savings Interest,Final Balance Savings,Initial Balance IRA,IRA Interest,Final Balance IRA,Total Expenses,Total Taxes,RMD Distribution
0,2023,62,580000.00,34800.00,551049.68,400000.00,32000.00,432000.00,59455.00,4295.32,0.00
1,2024,63,551049.68,33062.98,517971.02,432000.00,34560.00,466560.00,62089.50,4052.14,0.00
2,2025,64,517971.02,31078.26,481530.01,466560.00,37324.80,503884.80,63745.00,3774.28,0.00
3,2026,65,481530.01,28891.80,442009.64,503884.80,40310.78,544195.58,64944.00,3468.17,0.00
4,2027,66,442009.64,26520.58,399322.51,544195.58,43535.65,587731.23,66071.50,3136.20,0.00
5,2028,67,399322.51,23959.35,353299.74,587731.23,47018.50,634749.73,67204.50,2777.63,0.00
6,2029,68,353299.74,21197.98,303697.68,634749.73,50779.98,685529.71,68409.00,2391.04,0.00
7,2030,69,303697.68,18221.86,250265.10,685529.71,54842.38,740372.08,69646.50,2007.94,0.00
8,2031,70,250265.10,15015.91,192734.26,740372.08,59229.77,799601.85,70895.00,1651.75,0.00
9,2032,71,192734.26,11564.06,130882.77,799601.85,63968.15,863570.00,72143.50,1272.05,0.00


Important condsiderations:
- The cashflow stops the last year where you have IRA funds, regardless the future age you inputed above. 
- When the `Final Balance Savings` is less than `Total expenses` then withdrawls from the IRA begin. It does not leave the savings account in 0, so you can keep earning interests.
- When `RMD distribution` is biger than `Total expenses` for that year then the model considers `RMD distribution` as the amount that is withdrawn from the IRA.
- Total Taxes are substracted from `Final Balance Savings` or `Final Balance IRA` depending on which has balance. 


<br> 
If you would like reach a certain age higher than the one shown, you should consider reducing your annual expenses.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; **Remember** that if you want to try different values you just need to go to _Cell > Run All_
<br> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Hope this tool is useful for you, your husband and friends! It was a pleasure helping you. All the best!